# LangSmith 데이터셋 생성

자체 RAG 평가용 데이터셋을 구축하는 방법에 대해 알아보겠습니다.

먼저, 데이터셋 구축을 위해서는 크게 3가지 과정에 대한 이해가 필요합니다.

Case: Retrieval 이 Question 에 Relevant 한지 평가

> Question - Retrieval

![](./assets/eval-03.png)

Case: Answer 이 Question 에 Relevant 한지 평가

> Question - Answer   

![](./assets/eval-04.png)

Case: Answer 가 Retrieval 된 문서 안에서 답변하였는지 (Hallucination Check)

> Retrieval - Answer

![](./assets/eval-05.png)

따라서, `Question`, `Retrieval`, `Answer` 의 3가지 정보가 필요한 것이 일반적이지만, `Retrieval` 에 대한 Ground Truth 구축이 사실상 어렵습니다.

만약, `Retrieval` 에 대한 Ground Truth 가 존재한다면, 모두 데이터셋으로 저장하여 활용하고, 그렇지 않다면 `Question`, `Answer` 만으로 데이터셋을 구축하여 활용할 수 있습니다.

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


## 데이터셋 생성

`inputs` 와 `outputs` 를 활용하여 데이터셋을 생성합니다.

데이터셋은 `question` 과 `answer` 로 구성됩니다.

In [3]:
import pandas as pd

# 질문과 답변 목록
inputs = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행정명령을 발표한 날은 언제인가요?",
    "코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.",
]

# 질문에 대한 답변 목록
outputs = [
    "삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.",
    "2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.",
    "코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스 상태를 추적하고 투명성을 확보하기 위한 플랫폼입니다. 12개 기관과 협력하여 2,000여 개 데이터셋의 출처 정보를 제공하며, 개발자들이 데이터의 구성과 계보를 쉽게 파악할 수 있게 돕습니다.",
]

# 질문과 답변 쌍 생성
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]

# 데이터프레임으로 변환
df = pd.DataFrame(qa_pairs)

# 데이터프레임 출력
df.head()

,question,answer
0,삼성전자가 만든 생성형 AI의 이름은 무엇인가요?,삼성전자가 만든 생성형 AI의 이름은 삼성 가우스 입니다.
1,미국 바이든 대통령이 안전하고 신뢰할 수 있는 AI 개발과 사용을 보장하기 위한 행...,2023년 10월 30일 미국 바이든 대통령이 행정명령을 발표했습니다.
2,코히어의 데이터 출처 탐색기에 대해서 간략히 말해주세요.,코히어의 데이터 출처 탐색기는 AI 모델 훈련에 사용되는 데이터셋의 출처와 라이선스...


혹은 이전의 튜토리얼에서 생성한 Synthetic Dataset 을 활용할 수 있습니다.

아래 코드는 업로드한 HuggingFace Dataset 을 활용하는 예시입니다.

(참고) 아래의 주석을 풀고 실행하여 `datasets` 라이브러리를 업데이트 후 진행해 주세요. 

In [ ]:
# !pip install -qU datasets

In [4]:
import pandas as pd
from datasets import load_dataset, Dataset
import os

# huggingface Dataset에서 repo_id로 데이터셋 다운로드
dataset = load_dataset(
    "teddylee777/rag-synthetic-dataset",  # 데이터셋 이름
    token=os.environ["HUGGINGFACEHUB_API_TOKEN"],  # private 데이터인 경우 필요합니다.
)

# 데이터셋에서 split 기준으로 조회
huggingface_df = dataset["korean_v1"].to_pandas()
huggingface_df.head()

,contexts,evolution_type,metadata,episode_done,question,ground_truth
0,['∙ 3개국은 기반모델 전반에 대한 규제가 기술 중립적이고 위험 기반의 AI 규제...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,EU 규정의 맥락에서 AI 기반 모델과 관련하여 주요 고려 사항은 무엇인가요?,EU 규정의 맥락에서 AI 기반 모델과 관련된 주요 고려 사항으로는 의무적 자율 규...
1,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,"성능 지표, 아키텍처 설계 및 애플리케이션 영역의 변경 사항을 포함하여 이전 버전과...",이전 버전과 비교한 LLM 'Tongyi Qianwen 2.0'의 주요 기능 및 개...
2,['Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(C...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AAAI 컨퍼런스의 목적은 무엇인가요?,"AAAI 컨퍼런스의 목적은 연구자와 실무자가 모여 인공지능의 응용, 방법론, 미래 ..."
3,"['£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\n...",simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AI 개발의 맥락에서 콘텐츠 조정은 어떤 역할을 하나요?,콘텐츠 조정은 AI 시스템이 적절한 데이터로 학습하고 유해하거나 부적절한 콘텐츠가 ...
4,['1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n미국 연방거래...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True,AI 보상 및 소비자 보호에 대한 FTC의 우려는 무엇인가요?,FTC는 AI 관련 경쟁 및 소비자 보호 문제에 대한 우려를 제기했습니다. 특히 A...


## LangSmith 테스트를 위한 Dataset 생성

- `Datasets & Testing` 에 새로운 데이터셋을 생성합니다.

![](./assets/eval-06.png)

csv 파일에서 LangSmith UI를 사용하여 직접 데이터셋을 생성할 수도 있습니다.

자세한 사항은 아래 문서를 참고해 주세요.

- [LangSmith UI 문서](https://docs.smith.langchain.com/evaluation/faq/manage-datasets)

In [5]:
from langsmith import Client

client = Client()
dataset_name = "RAG_EVAL_DATASET"


# 데이터셋 생성 함수
def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset

    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset


# 데이터셋 생성
dataset = create_dataset(client, dataset_name)

# 생성된 데이터셋에 예제 추가
client.create_examples(
    inputs=[{"question": q} for q in df["question"].tolist()],
    outputs=[{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id,
)

데이터셋에 예제를 나중에 추가할 수 있습니다.

In [6]:
# 새로운 질문 목록
new_questions = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?",
]

# 새로운 답변 목록
new_answers = [
    "삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.",
    "사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자하기로 했습니다.",
]

# UI에서 업데이트된 버전 확인
client.create_examples(
    inputs=[{"question": q} for q in new_questions],
    outputs=[{"answer": a} for a in new_answers],
    dataset_id=dataset.id,
)

축하합니다! 이제 데이터셋이 준비되었습니다.